<a href="https://colab.research.google.com/github/MahdiTheGreat/Game-playing-systems/blob/main/AI_TTT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
!pip install -Uqq ipdb
import ipdb

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 16.2 MB/s eta 0:00:00
Automatic pdb calling has been turned ON


In [22]:
%pdb off

Automatic pdb calling has been turned OFF


In [45]:

 # MCTS self-play

import numpy as np
import random
import time
from math import sqrt, log
from copy import deepcopy

class MCTSNode:
    def __init__(self, state, symbol,parent=None):
        self.state = state  # Tic-Tac-Toe board state
        self.parent = parent
        self.children = []
        self.visits = 0
        self.value = 0
        self.untried_moves = self.get_legal_moves()
        self.symbol = symbol

    def get_legal_moves(self):
        return [(i, j) for i in range(3) for j in range(3) if self.state[i, j] == ""]

    def best_child(self, exploration_weight=1.41):
        return max(self.children, key=lambda child: child.value / (child.visits + 1e-6) + exploration_weight * sqrt(log(self.visits + 1) / (child.visits + 1e-6)))

    def expand(self):
        if not self.untried_moves:
            return self  # Return self if no moves left to expand
        move = self.untried_moves.pop()
        #new_state = self.state.copy()
        new_state = deepcopy(self.state)
        #new_state[move] = "X" if np.count_nonzero(self.state == "X") <= np.count_nonzero(self.state == "O") else "O"
        new_state[move] = self.symbol
        child_node = MCTSNode(state=new_state,symbol=self.symbol,parent=self)
        self.children.append(child_node)
        return child_node

    def update(self, result):
        self.visits += 1
        self.value += result


class MCTS:
    def __init__(self, symbol, time_limit=5.0, max_iterations=1000):
        self.time_limit = time_limit
        self.max_iterations = max_iterations
        self.symbol = symbol

    def search(self, root):
        start_time = time.time()
        iterations = 0

        #if root.untried_moves:
        #    root = root.expand()  # Ensure root expands at least once

        #while time.time() - start_time < self.time_limit and iterations < self.max_iterations:
        node = self.select(root)
        if node.untried_moves:
            node = node.expand()
        result = self.simulate(node.state)
        self.backpropagate(node, result)
        iterations += 1

        #return max(root.children, key=lambda c: c.visits, default=root)  # Choose best move by visits
        return root.best_child()  # Choose best move by value

    def select(self, node):
        # here we use a tree policy for selection
        while node.children and not node.untried_moves:
            node = node.best_child()
        return node

    def check_winner(self, state):
     for player in ["X", "O"]:
      for i in range(len(state)):
          if all(state[i, :] == player) or all(state[:, i] == player):
              return player
      if all(state.diagonal() == player) or all(np.fliplr(state).diagonal() == player):
          return player
     return None

    def simulate(self, state):
        state=deepcopy(state)
        #current_player = "X" if np.count_nonzero(state == "X") <= np.count_nonzero(state == "O") else "O"
        current_player = self.symbol
        while True:
            legal_moves = [(i, j) for i in range(len(state)) for j in range(len(state[0])) if state[i, j] == ""]
            if not legal_moves:
                return 0  # Draw
            move = random.choice(legal_moves)
            state[move] = current_player
            winner=self.check_winner(state)
            if winner==current_player:
                return 1 if current_player == "X" else -1
            current_player = "O" if current_player == "X" else "X"

    def backpropagate(self, node, result):
        while node:
            node.update(result)
            result = -result  # Alternate win/loss perspective
            node = node.parent

In [48]:
# MCTS vs user, doesnt work yet

import numpy as np
import random
import time
from math import sqrt, log
from IPython.display import clear_output
from google.colab import output

class TicTacToe:
    def __init__(self,human_player = "X",current_player="X"):
        self.state = np.full((3, 3), "")

        if human_player == "X":
            self.human_player = "X"
            self.ai_player2 = "O"
        elif human_player == "O":
            self.human_player = "O"
            self.ai_player2 = "X"
        else:
            self.ai_player = "X"
            self.ai_mcts = MCTS(self.ai_player)
            self.ai_player2= "O"
            self.human_player = None

        self.current_player = current_player
        self.ai2_mcts = MCTS(self.ai_player2)

    def display_board(self):
        print("  0 1 2")
        for i in range(len(self.state)):
            print(f"{i} {self.state[i][0]}|{self.state[i][1]}|{self.state[i][2]}")
            if i < 2:
                print("  -+-+-")
        print("\n")

    def get_human_move(self):
     while True:
      try:
          row = int(input(f"Enter row (0-{len(self.state)-1}): "))
          col = int(input(f"Enter column (0-{len(self.state[0])-1}): "))
          if self.state[row, col] == "":
              self.state[row, col] = self.human_player
              self.current_player = self.ai_player2  # Switch turn
              break
          else:
              print("Invalid move, try again!")
      except (ValueError, IndexError):
          print("Invalid input!")

    def get_ai_move(self,mcts):
     # AI's turn (MCTS)
     print("AI is thinking...")
     time.sleep(1)
     root = MCTSNode(state=deepcopy(self.state),symbol=mcts.symbol)
     best_move = mcts.search(root)
     #ipdb.set_trace()
     self.state = best_move.state
     self.current_player = self.human_player  # Switch turn

    def play_game(self):
        while True:
            self.display_board()

            def first_ai_player():
              self.get_ai_move(self.ai_mcts)

            first_player=self.human_player if self.human_player else self.ai_player
            first_player_move = self.get_human_move if self.human_player else first_ai_player

            if self.current_player == first_player:
              first_player_move()
            else:
              self.get_ai_move(self.ai2_mcts)

            winner=self.ai2_mcts.check_winner(self.state)
            if winner:
                self.display_board()
                print(f"{winner} wins!")
                break
            elif "" not in self.state:
                self.display_board()
                print("It's a draw!")
                break

if __name__ == "__main__":
    human_player = input("Choose a player (X or O): ")
    game = TicTacToe(human_player=human_player)
    game.play_game()


Choose a player (X or O): h
  0 1 2
0 ||
  -+-+-
1 ||
  -+-+-
2 ||


AI is thinking...
  0 1 2
0 ||
  -+-+-
1 ||
  -+-+-
2 ||X


AI is thinking...
  0 1 2
0 ||
  -+-+-
1 ||
  -+-+-
2 |O|X


AI is thinking...
  0 1 2
0 ||
  -+-+-
1 ||
  -+-+-
2 O|O|X


AI is thinking...
  0 1 2
0 ||
  -+-+-
1 ||O
  -+-+-
2 O|O|X


AI is thinking...
  0 1 2
0 ||
  -+-+-
1 |O|O
  -+-+-
2 O|O|X


AI is thinking...
  0 1 2
0 ||
  -+-+-
1 O|O|O
  -+-+-
2 O|O|X


O wins!
